Softmax与交叉熵损失
===

# 1.线性分类器
假设现在我们有一张2×2×1的图像(即图像大小为2×2，通道数为1)，像素值是$\begin{bmatrix}
56&231 \\
24&2
\end{bmatrix}$,此时我们构建一个线性分类期去判断该幅图像属于哪一种类别
![images](Images/02/13_03_001.png)

用一张图表示就是

![images](Images/02/13_03_002.png)

此时增加一下难度，如果我们的训练数据是64×64×3的RGB图像呢？同理，这时我们的$x$就是一个$12288 \times 1$的向量$64 \times 64 \times 3=12288$

![images](Images/02/13_03_003.png)

模型的参数可以通过最小化损失函数来获得

# 2.Sigmoid与Softmax的区别与联系
如果使用线性回归模型进行分类的话，得到的是一个具体的数值，并不直观。我们更希望可以直接得到某一样本属于各个类别的概率。对于二分类任务来说，我们可以采用对数几率回归来输出样本属于正类的概率。那么对于多分类该怎么办呢？答案就是softmax分类器。

## 2.1.Sigmoid函数
Sigmoid函数在[逻辑回归](02.机器学习.03.逻辑回归.ipynb)中已经介绍过了，它的形式是
$$S(x)=\frac{1}{1+e^{-x}}$$
Sigmoid是一个可微的有界函数，在各点均有非负的导数。当$𝑥 \to \infty$时，$𝑆(𝑥) \to 1$;当 $𝑥 \to -\infty$时，$𝑆(𝑥) \to 0$。常用于二元分类(Binary Classification)问题，以及神经网络的激活函数(Activation Function)(把线性的输入转换为非线性的输出)。

## 2.2.Softmax函数
Softmax主要用于多分类以及神经网络的概率输出。它的形式如下：
$$\sigma_i(Z)=\frac{e^{z_i}}{\sum_{k=1}^Ke^{z_k}}, j=1,2,...,K$$
![image](Images/02/13_03_004.jpg)

对于一个长度为$K$的任意实数矢量，Softmax可以把它压缩为一个长度为$K$的、取值在$(0, 1)$区间的实数矢量，且矢量中各元素之和为$1$。它在多元分类(Multiclass Classification)和神经网络中也有很多应用。Softmax不同于普通的 $max$函数：$max$函数只输出最大的那个值，而Softmax则确保较小的值也有较小的概率，不会被直接舍弃掉，是一个比较"Soft"的"max"。

In [1]:
import numpy as np
prediction = np.array([0.02760797,-0.00083022,0.00351067,-0.00328555,0.00372311,-0.00814351,-0.00122638,-0.00717432,0.00035223,0.00667405])
prediction = np.reshape(prediction, newshape=(1, 10))
exp_prediction = np.zeros(prediction.shape)
softmax = np.zeros(prediction.shape)
for i in range(prediction.shape[0]):
    prediction[i, :] -= np.max(prediction[i, :])
    exp_prediction[i] = np.exp(prediction[i])
    softmax[i] = exp_prediction[i]/np.sum(exp_prediction[i])
print(softmax)

[[0.10257674 0.09970073 0.10013446 0.09945623 0.10015573 0.09897425
  0.09966124 0.09907022 0.09981869 0.10045172]]


上述算法实现了Softmax的过程。prediction是神经网络预测出来的当前图片所对应的分类的概率。我们将prediction里面的每个值都减去分类概率的最大值，这样并不会影响最终的结果，但是却能使得我们计算exp时遇到上下溢的情况后表现的更为正常。这是因为对一个比较大的数求exponential非常容易发生overflow。求exponential之前将$z$的每一个元素减去$z_i$的最大值。这样求exponential的时候会碰到的最大的数就是0了，不会发生overflow的问题，但是如果其他数原本是正常范围，现在全部被减去了一个非常大的数，于是都变成了绝对值非常大的负数，所以全部都会发生underflow，但是underflow的时候得到的是0，这其实是非常meaningful的近似值，而且后续的计算也不会出现奇怪的NaN.最关键的是最后我们一般都会做normalization，所以同时减去一个任意数是不会改变最终的预测的


## 2.3.Sigmoid和Softmax的关系

对于Sigmoid来说，有
$$
\begin{split}
p(y=1|x)&=\frac{1}{1+e^{-\theta^Tx}} \\
p(y=0|x)&=1-p(y=1|x)=\frac{e^{-\theta^Tx}}{1+e^{-\theta^Tx}}
\end{split}
$$

对于$K=2$的Softmax来说，有
$$
\begin{split}
p(y=1|x)&=\frac{e^{\theta_1^Tx}}{e^{\theta_0^Tx}+e^{\theta_1^Tx}}=\frac{1}{1+e^{(\theta_0^T – \theta_1^T)x}}=\frac{1}{1+e^{-\beta x}} \\
p(y=0|x)&=\frac{e^{\theta_0^Tx}}{e^{\theta_0^Tx}+e^{\theta_1^Tx}}=\frac{e^{(\theta_0^T-\theta_1^T)x}}{1+e^{(\theta_0^T-\theta_1^T)x}}=\frac{e^{-\beta x}}{1+e^{-\beta x}} 
\end{split}
$$
其中$\beta=-(\theta_0^T-\theta_1^T)$,可见在二元分类的情况下，Softmax退化为了Sigmoid。

# 3.使用分类错误率与均方误差来解决分类问题

## 3.1.例子
根据一个人的年龄、性别、年收入等相互独立的特征，来预测一个人的政治倾向，有三种可预测结果：民主党、共和党、其他党。假设我们当前有两个逻辑回归模型（参数不同），这两个模型都是通过sigmoid的方式得到对于每个预测结果的概率值

模型一

| COMPUTED | TARGETS | CORRECT |
| -------- | ------- | ------- |
| 0.3 0.3 0.4 | 0 0 1(民主党) | 正确 |
| 0.3 0.4 0.3 | 0 1 0(共和党) | 正确 |
| 0.1 0.2 0.7 | 1 0 0(其它党) | 错误 |

模型二

| COMPUTED | TARGETS | CORRECT |
| -------- | ------- | ------- |
| 0.1 0.2 0.7 | 0 0 1(民主党) | 正确 |
| 0.1 0.7 0.2 | 0 1 0(共和党) | 正确 |
| 0.3 0.4 0.3 | 1 0 0(其它党) | 错误 |

## 3.2.分类错误率
$$\text{分类错误率}=\frac{\text{错误的数量}}{\text{总样本数量}}$$
模型一的分类错误率是$\frac{1}{3}$，模型二的分类错误率也是$\frac{1}{3}$。我们知道，模型1和模型2虽然都是预测错了1个，但是相对来说模型2表现得更好，损失函数值照理来说应该更小，但是，很遗憾的是，分类错误率并不能判断出来，所以这种损失函数虽然好理解，但表现不太好。

## 3.3.均方误差-Mean Squared Error
$$MSE=\frac{1}{n}\sum_i^n(\hat{y_i}-y_i)^2$$

模型一
$$
\begin{split}
L_1&=\frac{(0.3-0)^2+(0.3-0)^2+(0.4-1)^2}{3}=0.18 \\
L_2&=\frac{(0.3-0)^2+(0.4-1)^2+(0.3-0)^2}{3}=0.18 \\
L_3&=\frac{(0.1-1)^2+(0.2-0)^2+(0.7-0)^2}{3}=0.44 \\
\end{split}
$$
所有样本的loss平均为
$$MSE=\frac{0.18+0.18+0.44}{3}=0.26$$

模型二
$$
\begin{split}
L_1&=\frac{(0.1-0)^2+(0.2-0)^2+(0.7-1)^2}{3}=0.046 \\
L_2&=\frac{(0.1-0)^2+(0.7-1)^2+(0.2-0)^2}{3}=0.046 \\
L_3&=\frac{(0.3-1)^2+(0.4-0)^2+(0.3-0)^2}{3}=0.240 \\
\end{split}
$$
所有样本的loss平均为
$$MSE=\frac{0.046+0.046+0.24}{3}=0.11$$

MSE能够判断出来模型2优于模型1，那为什么不采样这种损失函数呢？主要原因是逻辑回归配合MSE损失函数时，采用梯度下降法进行学习时，会出现模型一开始训练时，学习速率非常慢的情况。有了上面的直观分析，我们可以清楚的看到，对于分类问题的损失函数来说，分类错误率和均方误差损失都不是很好的损失函数

# 4.交叉熵损失函数
$$H_p(q)=\sum_xq(x)ln\frac{1}{p(x)}$$
$q(x)$是真实分布，$p(x)$是预测分布

## 4.1.二分类
在二分的情况下，模型最后需要预测的结果只有两种情况，对于每个类别我们的预测得到的概率为$p$和$1-p$，有
$$
\begin{split}
H_p(q)&=\sum_xq(x)ln\frac{1}{p(x)} \\
&=q(x_1)ln\frac{1}{p(x_1)} + q(x_2)ln\frac{1}{p(x_2)} \\
&=-q(x_1)lnp(x_1)-q(x_2)lnp(x_2) \\
&=-[q(x_1)lnp(x_1)+q(x_2)log_2p(x_2)] \\
&=-[q(x_1)lnp+q(x_2)ln(1-p)] \\
&=-[ylnp+(1-y)ln(1-p)]
\end{split}
$$
其中y是样本的label，正类为1，负类为0

## 4.2.多分类
多分类就是使用的原始的公式

## 4.3.使用交叉熵损失来计算例子

模型一
$$
\begin{split}
L_1&=-(0 \times log_e{0.3} + 0 \times log_e{0.3} + 1 \times log_e{0.4})=0.91 \\
L_2&=-(0 \times log_e{0.3} + 1 \times log_e{0.4} + 0 \times log_e{0.3})=0.91 \\
L_3&=-(1 \times log_e{0.1} + 0 \times log_e{0.2} + 0 \times log_e{0.7})=2.30 \\
\end{split}
$$
求平均之后是
$$MSE=\frac{0.91+0.91+2.3}{3}=1.37$$

模型二
$$
\begin{split}
L_1&=-(0 \times log_e{0.1} + 0 \times log_e{0.2} + 1 \times log_e{0.7})=0.35 \\
L_2&=-(0 \times log_e{0.1} + 1 \times log_e{0.7} + 0 \times log_e{0.2})=0.35 \\
L_3&=-(1 \times log_e{0.3} + 0 \times log_e{0.4} + 0 \times log_e{0.4})=1.20 \\
\end{split}
$$
求平均之后是
$$MSE=\frac{0.35+0.35+1.2}{3}=0.63$$

可以发现，交叉熵损失函数可以捕捉到模型1和模型2预测效果的差异。

## 4.4.学习过程
由于交叉熵涉及到计算每个类别的概率，所以交叉熵几乎每次都和sigmoid(或softmax)函数一起出现。
- 神经网络最后一层得到每个类别的得分scores；
- 该得分经过sigmoid(或softmax)函数获得概率输出；
- 模型预测的类别概率输出与真实类别的one hot形式进行交叉熵损失函数的计算。